In [2]:
import cv2 as cv
import pickle
import numpy as np 

In [95]:


width,height=107,48
posList=[]  

with open('CarPark','rb') as f:
        posList=pickle.load(f)



def checkParkingSpace(imgProcess):
   counter=0
   for pos in posList:
        x,y=pos
        imgCrop=imgProcess[y:y+height,x:x+width]
        pexles=cv.countNonZero(imgCrop)
        if pexles<=800:
            color=(0,255,0)
            counter+=1
        elif pexles>800:
            color=(255,0,255)
        cv.putText(img,str(int(pexles)),(x+2,y+height-6),cv.FONT_HERSHEY_DUPLEX,0.5,color,1)
        cv.rectangle(img,(x,y),(x+width,y+height),color,3)
   
   cv.putText(img,str(f'Total Space: {len(posList)}'),(10,30),cv.FONT_HERSHEY_DUPLEX,1,(0,255,0),2)
   cv.putText(img,str(f'Empty Space: {counter}'),(10,70),cv.FONT_HERSHEY_DUPLEX,1,(0,255,0),2)
      #   cv.imshow(f'{i}',imgProcess)




totalSpace=len(posList)
cap=cv.VideoCapture('carPark.mp4')
while True:
   if cap.get(cv.CAP_PROP_POS_FRAMES)==cap.get(cv.CAP_PROP_FRAME_COUNT):
      cap.set(cv.CAP_PROP_POS_FRAMES,0)
   success,img=cap.read() 

   #Image Processing
   imgGray=cv.cvtColor(img,cv.COLOR_BGR2GRAY)
   imgBlur=cv.GaussianBlur(imgGray,(3,3),1)
   imgThreshold=cv.adaptiveThreshold(imgBlur,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C,cv.THRESH_BINARY_INV,25,16)
   #to remove the noise 
   imgMedianBlur=cv.medianBlur(imgThreshold,5)
   #making the white area a little bit thicker to make it easily detectable
   kernel=np.ones((3,3),np.uint8)
   imgDilate=cv.dilate(imgMedianBlur,kernel,iterations=1)


   for pos in posList:
        x,y=pos
        cv.rectangle(img,(x,y),(x+width,y+height),(255,0,255),2)

   # print(len(posList))
   checkParkingSpace(imgDilate)
   # cv.imshow('Thresh',imgThreshold)
   cv.imshow('Video',img)

   if cv.waitKey(10) & 0xFF==ord('d'):
      break
   

cv.destroyAllWindows()